In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  3 13:54:51 2021

@author: spi-2017-12
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  3 00:55:54 2021

@author: spi-2017-12
"""
import matplotlib.pyplot as plt
import matplotlib
from read_file import *
from alk import *


class Apasch(object):
    
    def __init__(self,data=[],salinity=38,DATA_DIR='',apasch_files=[''],campagne='campagne',CTE_APASCH={},head=['']):
        self.head=head
        self.data=data
        self.salinity=salinity
        self.apasc_file=apasch_files
        cwd = os.getcwd()
        DATA_DIR = os.path.join(cwd)
        self.DATA_DIR=DATA_DIR
        colonnes_alc=['CYCLE','DATE+HEURE','TEMP','A591','A437',"R'",
                                       'BLANC T591','BLANC T434','BLANC T810',
                                       'MESURE T591','MESURE T434','MESURE T810',
                                       'NH','NL','NTH(ALC)','RTH(ALC)','Alc 1',
                                       'Alc 2','Alc 3','Alc Sb']
        colonnes_pH=['Date','Temp','pH','Abs_LED3','Abs_LED2','Abs_LED1','R_indic']
        
        self.data_calcule=pd.DataFrame(columns=colonnes_pH)
        self.data_alk_calcule=pd.DataFrame(columns=colonnes_alc)
        self.campagne=campagne
        self.CTE_APASCH = CTE_APASCH
        
    def __str__(self):
        msg = 'Class APASCH, Data_Dir : ' + self.DATA_DIR
        return msg
    def __repr__(self):
        return self.__str__()

        
    
    def list_apasch_files(self,extension='TXT',dossier_apasch=''):
        FID = ReadFile()
        cwd = os.getcwd()
        CACHE_DIR = os.path.join(cwd,'cache')
        os.makedirs(CACHE_DIR, exist_ok=True)
        apasch_files = [x for x in os.listdir(self.DATA_DIR+'/'+dossier_apasch) if x.endswith('.'+extension)]
        print('Fichiers Apasch :' , apasch_files)
        self.apasch_files=apasch_files

    def file2dataframe(self,index=0):
        FID = ReadFile()
        data_pd = FID.readFid(['apasch_files/'+self.apasch_files[index]], self.CTE_APASCH)
        if self.CTE_APASCH["header"]==None :
            data_pd.columns = self.head
        self.data=data_pd
        data_pd
                


    
    def triplicate (self, data_pd) :
            for i in range(z,z+14,3) :#average the triplicate data:
                il=il+1;
                Ti.append(data_pd.iloc[i][10]);
                Avg_LED1.append((data_pd['LAMBDA_1'][i:i+3].mean()));
                Avg_LED2.append((data_pd['LAMBDA_2'][i:i+3].mean()));
                Avg_LED3.append((data_pd['LAMBDA_3'][i:i+3].mean()));
                Avg_Ref_LED1.append((data_pd['NL'][i:i+3].mean()));
                Avg_Ref_LED2.append((data_pd['Nthph'][i:i+3].mean()));
                Avg_Ref_LED3.append((data_pd['100_T_cel'][i:i+3].mean()));
        
        

    def calcul_pH(self,data_pd) :
        
        # Initialisation 
        # Salinité
        S=self.salinity;
        size_data=15
        Abs_LED1 = np.zeros((size_data//15,4))
        Abs_LED2 = np.zeros((size_data//15,4))
        Abs_LED3 = np.zeros((size_data//15,4))
        R_indic  = np.zeros((size_data//15,4))
        pHj  = []
        temp=[]
        datef=[]
        
        zl=0
        for z in range (0,15,15) : #1 measurement=15 lines
            zl+=1;
        # date of the sample
            #date_sample=data_pd.iloc[z][0]+' '+data_pd.iloc[z][1];
            date_sample=data_pd.iloc[z][0]
        # sample temperature= average of temperature during all the measurements of the sample    
            T=np.mean(data_pd['T_cel'][z:z+14])
            il=0 
            Ti=[]
            Avg_LED1=[]    
            Avg_LED2=[]
            Avg_LED3=[]
            Avg_Ref_LED1=[]    
            Avg_Ref_LED2=[]
            Avg_Ref_LED3=[]
            
            for i in range(z,z+14,3) :#average the triplicate data:
                il=il+1;
                Ti.append(data_pd.iloc[i][9]/100);
                Avg_LED1.append(data_pd.iloc[i:i+3,3].mean());
                Avg_LED2.append(data_pd.iloc[i:i+3,4].mean());
                Avg_LED3.append(data_pd.iloc[i:i+3,5].mean());
                Avg_Ref_LED1.append((data_pd.iloc[i:i+3,7].mean()));
                Avg_Ref_LED2.append((data_pd.iloc[i:i+3,8].mean()));
                Avg_Ref_LED3.append((data_pd.iloc[i:i+3,9].mean()));
        
           #calcul des absorbances pour chaque addition de colorant :
            for j in range (0,4) : 
              #  print(zl)
                Abs_LED1[zl-1][j]=np.log10(Avg_LED1[0]/Avg_LED1[j+1]);
                Abs_LED2[zl-1][j]=np.log10(Avg_LED2[0]/Avg_LED2[j+1])-Abs_LED1[zl-1][j];
                Abs_LED3[zl-1][j]=np.log10(Avg_LED3[0]/Avg_LED3[j+1])-Abs_LED1[zl-1][j];
                R_indic[zl-1][j]=Abs_LED2[zl-1][j]/Abs_LED3[zl-1][j]                     #rapport des absorbances
            #Calibration d'Apasch pour 4°C<T<31°C et 20<S<40
                Tki=273.15+Ti[4]; 
                e1=0.0170595;
                e4=1.560023*10**3+4.811158*10**(-1)*Tki+1.146155*10**(-1)*S-1.933045*10**(-3)*S*S-3.958658*10**4/Tki-2.75658*10**2*np.log(Tki)-2.026206*10**(-2)*np.log(Tki)*S+3.418927*10**(-4)*np.log(Tki)*S*S;       
            #Equation en utilisant la relation de Dickson pour le tris:
               # pK2e2=1.262784*10**2-4.640924*S+3.34405*10**(-3)*S*S-(4.154957*10**3)/Tki-(1.834135*10**1)*np.log(Tki)+(1.947547*10**2*S)/Tki+6.987689*10**(-1)*S*np.log(Tki)-5.741784*10**(-4)*S*S*np.log(Tki);
                pK2e2=1.17901*10**2-4.358595*S+4.262097*10**(-3)*S*S-(3.755133*10**3)/Tki-(1.71209*10**1)*np.log(Tki)+(1.800733*10**2*S)/Tki+6.585909*10**(-1)*S*np.log(Tki)-7.443867*10**(-4)*S*S*np.log(Tki)
            #Equation en utilisant la deuxième relation pour le tris:
            pHj.append(pK2e2+np.log10((R_indic[zl-1][0]-e1)/(1-R_indic[zl-1][0]*e4)));
            temp.append(Ti[4])
            datef.append(date_sample)
        # On arrange dans un dataframe
        temp=np.array(temp)
        temp.reshape(-1,1)
        df = pd.DataFrame({'Date': datef,'Temp':temp,'pH':pHj,'Abs_LED3':Abs_LED3[:,-1],'Abs_LED2':Abs_LED2[:,-1],'Abs_LED1':Abs_LED1[:,-1],'R_indic' :R_indic[:,-1]} )
  #      self.data_calcule = df.round(4)
        self.data_calcule = pd.concat([self.data_calcule,df.round(4)]
            , axis=0, join="inner",ignore_index=True)
        return True
    
    def calcul_alk_ed_cycle(self,un_cycle): 
        # moyennes du cycle
        moyennes=param_alk().calcul_moyenne (un_cycle)
        # calcul des differentes alcalinites
        alk=param_alk().caclul_alcalinites (un_cycle)
        # concatenation
        alc=pd.concat([moyennes, alk], axis=1, join="inner")
        self.data_alk_calcule=pd.concat([self.data_alk_calcule,alc]
            , axis=0, join="inner",ignore_index=True)
      
     
        
if __name__=="__main__":
    print('classe apasch  ok')
    # apa=Apasch()
    # apa.campagne='Moose 2019'
    # apa.list_apasch_files('TXT','apasch_files')
    # apa.head=['DATE','TIME','COUNT','TYPE','LED1','LED2','LED3','NH','Ref_LED1','Ref_LED2','Ref_LED3','T_cel']
    # apa.CTE_APASCH = {"header": None,
    #               "sep": '\s+',
    #               "ext": 'txt',
    #               "col": [],
    #               "t_format": '%Y/%m/%d %H:%M:%S',
    #               "t_name": 'DATE_TIME',
    #               "path": apa.DATA_DIR,
    #               }
    # # On transforme le premier fichier de donnees (indice 0 ) en dataframe  
    # apa.file2dataframe(0)
    # # On fait le calcul pour chaque cycle du fichier 
    # print("Caclul de l'alcalinité en eaux douces : ")
    # # un cycle tient sur 6 lignes
    # for j in range(0,len(apa.data)//6)  :
    #     un_cycle=apa.data[apa.data['COUNT']==j+1]
    #     apa.calcul_alk_ed_cycle(un_cycle)
    # # Ecriture sur un fichier txt des resultats
            
    # filename='catenoy_alk.txt'
    # sep='\t'
    # apa.data_alk_calcule.to_csv(filename,columns=['CYCLE','DATE+HEURE','TEMP','Alc 1','Alc 2','Alc 3','Alc Sb'],float_format='%.6f'
    #            ,sep = sep,index=None)  

classe apasch  ok


In [11]:
# -*- coding: utf-8 -*-

import yaml
#from apasch_class_alc import *
import Apasch_cycles
import UDP_Read
import time
import save_Data
import Apasch2data
import dataBuffer
import pandas as pd
# import only system from os
from os import system, name


class Main():

    def __init__(self):

        ###############_Init from conf file_###############
        yaml_content = yaml.safe_load(open("conf.yml", 'r'))
        self.TSG = yaml_content["TSG"]
        self.UDP = yaml_content["TSG"]
        self.NTP = yaml_content["NTP"]
        self.NMEA = yaml_content["NMEA"]
        self.GENERAL = yaml_content["GENERAL"]
        self.SAUVE = yaml_content["SAUVE"]
        self.CTE = yaml_content["CTE"]
        self.apasch = Apasch_cycles.Interface(yaml_content["RS232"])
        self.conv = Apasch2data.calc()
        self.data_cycle=pd.DataFrame([{key: '' for key in self.SAUVE["HEADER"]}])

    def clear_screen(self):
            # for windows
        if name == 'nt':
            _ = system('cls')
            # for mac and linux(here, os.name is 'posix')
        else:
            _ = system('clear') 
                
    def conf(self):

        #############_init UDP NMEA_########################
        if self.NMEA['is_active']:
            try:
                self.nav = UDP_Read.Udp(self.NMEA)
            except:
                print("connection error on UDP NMEA")
        else:
            print("\033[91m NMEA is not activate  \033[89m")

        #############_init TSG_#############################
        if self.TSG['is_active']:
            try:
                self.termosalino = UDP_Read.Udp(self.TSG)
            except:
                print("connection error on UDP TSG")

        if self.GENERAL["PH_ACTIVE"]:
            print('Voie pH activé')
            self.save_Raw_ph = save_Data.Save(PARAM=self.SAUVE, type="RAW",name="PH" )

        if self.GENERAL["ALC_ACTIVE"]:
            print('Voie Alcalinité activé')
            self.save_Raw_Alc = save_Data.Save(PARAM=self.SAUVE, type="RAW", name="ALC")

        return self.GENERAL

    def trame(self):
        if self.NMEA['is_active']:
            return self.nav.read_Udp()
        else:
            return {"NMEA": "nan"}

    def tsg(self):
        if self.TSG['is_active']:
            return self.termosalino.read_Udp()
        else:
            return {"TSG": ""}

    def modesingle(self, cmd):
        if "RINCE" in cmd:
            self.rince_cel(cmd)
        else:
            self.apasch.cmdsimple(cmd=cmd)
        return "message envoyé single"

    def rince_cel(self, cel):

        n = 0
        cmd = self.GENERAL[cel]

        try:

            while n < cmd["nb_seq"]:
                n += 1
                print("{} n° {} de la celulle".format(cmd["display"], n))
                for send in cmd["send"]:
                    print(" {} ---> send {}  ".format(time.strftime('%Y/%m/%d %H:%M:%S '), send))
                    self.apasch.cmdsimple(send)
                    if send == "K":
                        time.sleep(cmd["Tp"])
                    time.sleep(cmd["Tr"])
            print("{} ---> Rinçage fini".format(time.strftime('%Y/%m/%d %H:%M:%S ')))
        except:
            print(" {} ---> it seems like {} does not exist ".format(time.strftime('%Y/%m/%d %H:%M:%S '), cel))

        return time.strftime('%Y/%m/%d %H:%M:%S ')

 
    def cmd_simple(self, cmd):
        if cmd !="exit":
            try:
                send = self.GENERAL[cmd]['send']
                print(" {} ---> send {}  ".format(time.strftime('%Y/%m/%d %H:%M:%S '), send))
                return(self.apasch.send(send,tr=self.GENERAL[cmd]['Tr']))
            except:
                print(" {} ---> it seems like {} does not exist here ".format(time.strftime('%Y/%m/%d %H:%M:%S '),cmd))

                return "error"
            
    def update_read (self,read,count,key,cycle,data,CTE) :
        read.update({
                    "DATETIME": time.strftime('%Y/%m/%d %H:%M:%S '),
                    "COUNT": count,
                    "TYPE": key['display'],
                    })
        read.update(self.trame())
        read.update(self.tsg())
        read.update(self.conv.temperature(cycle,data,CTE))


    def cycle(self, cycle, sauve, count):
        for key in self.GENERAL[cycle]:
            it = 0
            while it < key["iteration"]:
                it+=1
                read = {key: '' for key in self.SAUVE["HEADER"]}
                for send in key["send"]:
                    self.clear_screen()
                    print("Cycle {} , {}_{} : ".format(count ,key['display'],it))
                    CTE=self.CTE["THERMISTANCE"]
                    cmd=send
                    commande_simple = self.cmd_simple(cmd)
                    self.update_read(read,count,key,cycle,commande_simple,CTE)
                    sauve.write(read)
                    self.data_cycle = self.data_cycle.append(read, ignore_index=True)    
   
    def close(self):
        pass

    
    def modeAuto_Ph(self, count , apa):
            self.cycle("PH1", self.save_Raw_ph, count)
            cycle_brut=self.data_cycle
            if count == 0 : cycle_brut.drop(0,0,inplace=True)
            apa.calcul_pH(cycle_brut)
            return apa.data_calcule
    
            
    def modeAuto_ALC(self, count,apa):

            self.cycle("ALC", self.save_Raw_Alc, count)
            cycle_brut=self.data_cycle
            if count == 0 : cycle_brut.drop(0,0,inplace=True)
            apa.calcul_alk_ed_cycle(cycle_brut)
            return apa.data_alk_calcule



if __name__ == '__main__':

    main = Main()
    main.conf()
    apa=Apasch()
    cmd = ''
    name = ''


    while cmd != "stop":
        cmd = input("enter mode (cycle/single/test/stop)? ")
        if cmd == "cycle":
            try:
                COUNT = 0
                while COUNT < 1 :
             #   while True:
                    if main.GENERAL["PH_ACTIVE"]:
                        pH = main.modeAuto_Ph(COUNT,apa)
                        print(pH)
                    if main.GENERAL["ALC_ACTIVE"]:
                        alc = main.modeAuto_ALC(COUNT,apa)
                        print(alc[['DATE+HEURE','CYCLE',
                                                        'TEMP','Alc 1','Alc 2','Alc 3','Alc Sb']])
                    COUNT += 1
            except KeyboardInterrupt:
                print("boucle interrompue")

        elif cmd == "single":
            while name != "exit":
                print(name)
                name = input("What is your command? or exit: ")
                sequence = main.cmd_simple(name)
        elif cmd == "test":
            print(main.trame())
            print(main.tsg())
        elif cmd == "stop":
            main.data_cycle

 Erreur ouverture port serie /dev/ttyUSB0 
 NMEA is not activate  
Voie Alcalinité activé
MOOSE_GE_LEG_1data RAW_ MOOSE_GE_LEG_1 0728-2021_051447 .txt
enter mode (cycle/single/test/stop)? cycle
Cycle 0 , Blanc_Alc_1 : 
 2021/07/28 05:14:49  ---> send B  
Cycle 0 , Blanc_Alc_2 : 
 2021/07/28 05:14:49  ---> send B  
Cycle 0 , Blanc_Alc_3 : 
 2021/07/28 05:14:49  ---> send B  
Cycle 0 , Mesure_Alc_1 : 
 2021/07/28 05:14:49  ---> send M  
Cycle 0 , Mesure_Alc_2 : 
 2021/07/28 05:14:49  ---> send M  
Cycle 0 , Mesure_Alc_3 : 
 2021/07/28 05:14:49  ---> send M  
Cycle 0 , Mesure_Alc_4 : 
 2021/07/28 05:14:49  ---> send M  


/home/spi-2017-12/Bureau/apasch (copie)/Apasch/apasch/alk.py:106: RuntimeWarning: invalid value encountered in double_scalars
  R_prime=A437/A591


               DATE+HEURE CYCLE    TEMP  Alc 1  Alc 2  Alc 3  Alc Sb
0 2021-07-28 05:14:48.999     0  0.2832    NaN    NaN    NaN     NaN
enter mode (cycle/single/test/stop)? stop


In [9]:
alc

,CYCLE,DATE+HEURE,TEMP,A591,A437,R',BLANC T591,BLANC T434,BLANC T810,MESURE T591,MESURE T434,MESURE T810,NH,NL,NTH(ALC),RTH(ALC),Alc 1,Alc 2,Alc 3,Alc Sb
0,0,2021-07-28 04:31:06.428,0.2832,0.0,0.0,NaN,5170.0,2949.0,772.0,5170.0,2949.0,772.0,3735.0,4587.0,4587.0,1958.0,NaN,NaN,NaN,NaN


In [8]:
main.data_cycle

,DATETIME,COUNT,TYPE,LAMBDA_1,LAMBDA_2,LAMBDA_3,NH,NL,Nth,100_T_cel,T_cel,TSG,NMEA
1,2021/07/28 04:31:06,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
2,2021/07/28 04:31:06,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
3,2021/07/28 04:31:06,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
4,2021/07/28 04:31:06,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
5,2021/07/28 04:31:07,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
6,2021/07/28 04:31:07,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
7,2021/07/28 04:31:07,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan


In [20]:
pH

,Date,Temp,pH,Abs_LED3,Abs_LED2,Abs_LED1,R_indic
0,2021/07/28 02:36:32,19.58,NaN,0.0,0.0,0.0,NaN
1,2021/07/28 02:36:32,19.58,NaN,0.0,0.0,0.0,NaN


In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  9 12:07:42 2021

@author: spi-2017-12
"""

import os, glob, json
from datetime import date, timedelta, datetime
import pandas as pd
from netCDF4 import Dataset
import numpy as np
import configparser
import matplotlib.pyplot as plt
import matplotlib


class param_alk(object):
    def __init__(self,param1=[],calib=[],epsilon=[]):
        self.param1 = self.parametres()
        self.calib = self.calibration()
        self.epsilons=self.calcul_epsilons()
        self.param_optiques=self.calcul_param_optiques()

    def parametres (self) :
        vpompe=5.37e-5
        hfo_cwa=2.1805e-1
        indic_cwa=2e-4 
        dilu_color=0.0063/(vpompe*4)
        dilu_ech= 0.0063/(0.0063-(vpompe*4))
        
        parametres = pd.DataFrame({'Volume pompe' : '{:,.7f}'.format(vpompe) , 'Dilution colorant': dilu_color, 
                               'Dilution echantillon':dilu_ech,
                               'hfo(cwa)' : hfo_cwa,'hfo(cellule)': hfo_cwa/dilu_color,
                               'indicateur(cwa)' :indic_cwa, 'indicateur(cellule)' :'{:,.8f}'.format(indic_cwa/dilu_color)
                              },index=[0])
        return parametres


    def calibration(self):
        calibrer=pd.DataFrame({'A434': pd.Series([0.0851,0.1,0.0064,0.0823], index = ['Acide', 'Alc ON', 'NaOH','eau'])
                      , 'A591': pd.Series([0.0005,0.0479,0.3164,0.1001], index = ['Acide', 'Alc ON', 'NaOH','eau'])})
        fit=np.polyfit([0.1,0.0064,0.0823], [0.0479,0.3164,0.1001], 1, rcond=None, full=False, w=None, cov=False)
        pente=fit[0]
        oo= fit[1]
        return calibrer,pente,oo
        
            
    
    def calcul_epsilons(self):
        parametres=self.param1
        ae_591 = 0.000480708
        ae_434 = ( ae_591 - self.calib[2] )/self.calib[1]
        eps437j = ae_434/float(parametres['indicateur(cellule)'])
        eps437b = self.calib[0]['A434']['NaOH'] /float(parametres['indicateur(cellule)'])
        eps591j = ae_591/float(parametres['indicateur(cellule)'])
        eps591b = self.calib[0]['A591']['NaOH'] /float(parametres['indicateur(cellule)'])
        
        epsilon = pd.DataFrame({'eps(437,J)': eps437j, 
                                 'eps(437,B)': eps437b,
                                 'eps(591,J)': eps591j,
                                 'eps(591,B)': eps591b,             
                              },index=[0])
        return epsilon
    
    def calcul_param_optiques (self) : 
        e1 = self.epsilons['eps(591,J)']/ self.epsilons['eps(437,J)']
        e2 = self.epsilons['eps(591,B)']/ self.epsilons['eps(437,J)']
        e3 = self.epsilons['eps(437,B)']/ self.epsilons['eps(437,J)']
        e4 = e3/e2
        s=self.calib[1]
        e2_sb = -s/(1-s*e4)
        B591 = self.calib[0]['A591']['NaOH'] 
        param_optiques = pd.DataFrame({'e1': e1, 
                                 'e2': e2,
                                 'e3': e3,
                                 'e4': e4,  
                                 's': s,  
                                 'e2 (Sb)': e2_sb,  
                                 'B591' : B591
                              },index=[0])
        return param_optiques
    
    
    
    def calcul_moyenne(self,un_cycle=[]):
        
        df=un_cycle
        # calcul des moyennes 
        CYCLE=int(df['COUNT'].mean())
        TEMP=df['T_cel'].mean()/100
        NH=df.iloc[:,6].mean()
        NL=df.iloc[:,7].mean()
        NTH=df.iloc[:,8].mean()
        RTH=df.iloc[:,9].mean()
        T810_blanc=df[df['TYPE']=='Blanc_Alc'].mean()['LED1']
        T810_mesure=df[df['TYPE']=='Mesur_Alc'].mean()['LED1']
        T434_blanc=df[df['TYPE']=='Blanc_Alc'].mean()['LED3']
        T434_mesure=df[df['TYPE']=='Mesur_Alc'].mean()['LED3']
        T591_blanc=df[df['TYPE']=='Blanc_Alc'].mean()['LED2']
        T591_mesure=df[df['TYPE']=='Mesur_Alc'].mean()['LED2']    
        # calcul des deux coeff avec log
        A591=( np.log10(T591_blanc/T591_mesure) - np.log10( T810_blanc/T810_mesure) ) /2.2
        A437=( np.log10( T434_blanc/T434_mesure)-np.log10( T810_blanc/T810_mesure) ) /2.2
        R_prime=A437/A591
        time= pd.to_datetime(df['DATE']+' '+df['TIME'], format='%d/%m/%Y %H:%M:%S')
        TIME=time.mean()
        TIME= pd.to_datetime(TIME, format='%d/%m/%Y %H:%M:%S')
        moy = pd.DataFrame({'CYCLE' : CYCLE ,'DATE+HEURE' : TIME,
                                       'TEMP' : TEMP ,'A591' : A591,'A437':A437,"R'":R_prime,
                                       'BLANC T591':T591_blanc,'BLANC T434':T434_blanc,'BLANC T810' : T810_blanc,
                                       'MESURE T591' : T591_mesure,'MESURE T434':T434_mesure,'MESURE T810':T810_mesure,
                                       'NH':NH,'NL':NL,'NTH(ALC)':NTH,'RTH(ALC)':RTH },index=[0])

        return moy 
      
    
    def caclul_alcalinites (self,un_cycle,pka_ind=3.89):
        param_optique=self.param_optiques
        calib=self.calib[0]
        parametres=self.param1
        moy=self.calcul_moyenne(un_cycle)
        B591=moy['A591']-(param_optique['s']*moy['A437'])/(1-param_optique['e3'])
        Dr1=((1/moy["R'"])-param_optique['e1'])/(param_optique['e2']-(1/moy["R'"])*(param_optique['e3']))
        Dr2=moy["R'"]*param_optique['e2']-param_optique['e3']
        Dr3a=Dr1
        Dr3b=Dr2
        Drsb=param_optique['e2 (Sb)']*(moy["R'"]-param_optique['e4'])
        # Calcul des pH
        pH1= pka_ind-np.log(Dr1)
        pH2=pka_ind-np.log(Dr2)
        pH3a = pka_ind-np.log(Dr3a)
        pH3b = pka_ind-np.log(Dr3b)
        pHsb = pka_ind-np.log(Drsb)
        # calcul du pKA
        pka_hfo=0.014*(moy["TEMP"]+273.15)+(1278.3/(moy["TEMP"]+273.15)-4.9285+0.08)
        #calcul de K
        k_ind=10**(-pka_ind)/10**(-pka_hfo)
        
        # calcul de l'alcalinité
        
        alc1=(((parametres["hfo(cellule)"]*((moy['A591']-param_optique['e1']*moy['A437'])/
             (param_optique['B591']-calib['A434']['NaOH']*param_optique['e1']))*
             ((1+(1/Dr1))/(1+(k_ind*(1/Dr1)))))-(10**(-pka_ind)*(1/Dr1)))*parametres['Dilution echantillon'])/(1-
                                                                             (0.0000003/(10**-pka_ind*(1/Dr1))))
        alc2 =(((parametres["hfo(cellule)"]*(moy['A591']/param_optique['B591'])*
              ((1+Dr2)/(1+(k_ind*Dr2))))-(10**(-pka_ind)*Dr2))
              *parametres['Dilution echantillon'])/(1-(0.0000003/(10**(-pka_ind)*Dr2)))
        
        alc3=1000000*(((parametres["hfo(cellule)"]*(moy['A591']/param_optique['B591'])
                    *((1+Dr3b)/(1+(k_ind*Dr3b))))-(10**-pka_ind*Dr3b))
                    *parametres['Dilution echantillon'])/(1-(0.0000003/(10**-pka_ind*Dr3b)))
        
        alcsb=(((parametres["hfo(cellule)"]*((1-param_optique['s']*param_optique['e4'])/(1-param_optique['s']*moy["R'"]))
               *((1+Drsb)/(1+(k_ind*Drsb))))-(10**-pka_ind*Drsb))
               *parametres['Dilution echantillon'])/(1-(0.0000003/(10**-pka_ind*Drsb)))
    
        alcalinites = pd.DataFrame({'Alc 1' : alc1 ,'Alc 2' : alc2,'Alc 3' : alc3,'Alc Sb' : alcsb},index=[0])
    
        return alcalinites
     
        

   
        
if __name__=="__main__":
    
    
    print('classe alcalinité eaux douces')

classe alcalinité eaux douces


In [8]:
k

,DATETIME,COUNT,TYPE,LAMBDA_1,LAMBDA_2,LAMBDA_3,NH,NL,Nth,100_T_cel,T_cel,TSG,NMEA
1,2021/07/25 23:40:44,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
2,2021/07/25 23:40:44,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
3,2021/07/25 23:40:44,0,Blanc_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
4,2021/07/25 23:40:44,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
5,2021/07/25 23:40:44,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
6,2021/07/25 23:40:44,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
7,2021/07/25 23:40:44,0,Mesure_Alc,772,5170,2949,3735,4587,4587,1958,28.32,,nan
